In [ ]:
import numpy as np    # math and array handling 
import pandas as pd   # some functions to load data and handle tables
import pylab as plt   # plotting stuff
from glob import glob # to search files 
import sys; sys.path.append('..') # this is not needed if you install the module
from chipmunk_performance_plots import *
%matplotlib widget

def performance_summary():
    fn = pick_files("*.mat")
    h5file = convert_specified_behavior_sessions(fn)
    session_data = pd.read_hdf(h5file[0])

    ntrials = len(session_data)
    ntrials_with_choice = len(session_data[session_data['outcome_record'].isin([0, 1])])
    stim_rates = np.array([len(timestamps) for timestamps in session_data.stimulus_event_timestamps])
    unique_stim_rates = np.unique(stim_rates)

    response = []
    for r in session_data.response_side:
        if r == 0: #left choice
            response.append(-1)
        elif r == 1: #right choice
            response.append(r)
        else: #no choice/early withdrawal
            response.append(0)

    fig = plt.figure(1,figsize=(12, 8))

    ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=1) 
    ax2 = plt.subplot2grid((3, 3), (0, 1), colspan=1)
    ax3 = plt.subplot2grid((3, 3), (0, 2), colspan=1)
    ax4 = plt.subplot2grid((3, 3), (1, 0), colspan=3)

    # First plot - response+noise per stimulus rate
    ax1.plot(stim_rates + np.random.uniform(-2, 2, len(session_data)),
             response + np.random.uniform(-0.1, 0.1, len(session_data)),
             'ko', alpha=0.1, markersize=2)
    
    ax1.set_ylabel('Response', fontsize=14)
    ax1.set_xlabel('Stimuli intensity (event rate)', fontsize=14)
    ax1.set_yticks([-1, 0, 1], ['left', 'no', 'right'])

    # Second plot - response color-coded by outcome throughout the session
    c = []
    for i, (s, r) in enumerate(zip(np.array(stim_rates), np.array(response))):
        if ((s > 12) & (r > 0)) | ((s < 12) & (r < 0)):
            c.append('green')
        elif ((s == 12) & (session_data.outcome_record[i] == 1)):
            c.append('green')
        elif ((s == 12) & (session_data.outcome_record[i] == 0)):
            c.append('red')
        elif r == 0:
            c.append('grey')
        else:
            c.append('red')

    ax4.scatter(np.arange(len(stim_rates)), stim_rates, 5, c=c, alpha=0.5)
    ax4.set_ylabel('Stim frequency', fontsize=14)
    ax4.set_yticks(np.unique(stim_rates))
    ax4.set_xlabel('Trial number', fontsize=14)

    # ax2.scatter([], [], 5, c='green', label='Correct', alpha=0.5)
    # ax2.scatter([], [], 5, c='red', label='Wrong', alpha=0.5)
    # ax2.scatter([], [], 5, c='grey', label='No response', alpha=0.5)
    # ax2.legend(loc='upper right', fontsize=8)


    # Third plot - histogram showing outcome record by trial type
    correct_responses = (session_data.outcome_record == 1)
    incorrect_responses = (session_data.outcome_record == 0)
    no_choice_responses = (session_data.outcome_record == -1)

    # Unique stim intensities
    correct = np.zeros_like(unique_stim_rates)
    incorrect = np.zeros_like(unique_stim_rates)
    no_choice = np.zeros_like(unique_stim_rates)
    # count the trials
    for i,rate in enumerate(unique_stim_rates):
        # print(i,rate)
        correct[i] = np.sum(correct_responses[np.array(stim_rates) == rate])
        incorrect[i] = np.sum(incorrect_responses[np.array(stim_rates) == rate])
        no_choice[i] = np.sum(no_choice_responses[np.array(stim_rates) == rate])

    # make a figure
    ax2.bar(unique_stim_rates,correct,width = 1,color = 'green', label='Correct')                # correct choices in BLACK
    ax2.bar(unique_stim_rates,incorrect,bottom = correct,width = 1,color='red', label='Incorrect') # incorrect choices in RED
    ax2.bar(unique_stim_rates,no_choice,bottom = correct+incorrect,width = 1,color='gray', label='Early Withdrawal') # no choice in GRAY
    ax2.set_xlabel('Stimuli intensity (event rate)', fontsize = 14) # set the x-axis label
    ax2.set_ylabel('Number of trials', fontsize = 14) # set the y-axis label
    ax2.set_xticks([*unique_stim_rates]);
    ax2.legend(loc='upper right', fontsize = 8)

    # Fourth plot - psychometric points + 95% CI
    sel = session_data[session_data.response_side.isin([0,1])] # select only trials where the subject responded
    responded_right = np.array(sel.response_side == 1).astype(int) # select the response = 1 (i.e. the left side) and cast to integer datatype (number)   
    sel_stim_rates = np.array([len(timestamps) for timestamps in sel.stimulus_event_timestamps])

    # This is the function that does the actual computing, the rest is trial selection and plotting
    stims, p_side, ci_side, n_obs, n_side = compute_proportions(stim_values=sel_stim_rates, response_values=responded_right)

    ax3.vlines(12,0,1,color = 'k',lw = 0.3) # plot a vertical line as reference at zero
    ax3.hlines(0.5,np.min(stims),np.max(stims),color = 'k',lw = 0.3) # plot an horizontal line as reference for chance performance

    # plot the observed data and confidence intervals
    for i,e in zip(stims,ci_side):  # plot the confidence intervals
        ax3.plot(i*np.array([1,1]),e,'_-',lw=0.5,color = 'black')
    ax3.plot(stims,p_side,'ko',markerfacecolor = 'lightgray',markersize = 6)

    ax3.set_ylabel('P$_{right}$',fontsize = 18)  # set the y-axis label with latex nomenclature
    ax3.set_xlabel('Stimulus rate (Hz)', fontsize = 14); # set the x-axis label


    fig.tight_layout()
    plt.show()

performance_summary()




